<a href="https://colab.research.google.com/github/shrutishrinivasan/CIA_DAA/blob/main/CulturalAlgo(NN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
data = pd.read_csv("loan.csv")
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [30]:
df = data
# removing irrelevant columns like Id, Zip code
# and shifting the target variable to the last position
df = df.drop(['ID','ZIP Code','Personal Loan'],axis=1)
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,1,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1


In [31]:
# Target variable: Personal Loan
df['Perosnal Loan'] = data['Personal Loan']
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Perosnal Loan
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0


In [32]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.25)
X_train

array([[ 51.,  25., 163., ...,   0.,   1.,   0.],
       [ 42.,  17.,  44., ...,   1.,   1.,   1.],
       [ 35.,  11.,  24., ...,   0.,   0.,   0.],
       ...,
       [ 26.,   1.,  24., ...,   0.,   0.,   1.],
       [ 57.,  33.,  49., ...,   1.,   1.,   1.],
       [ 33.,   9.,  38., ...,   0.,   0.,   0.]])

In [34]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [36]:
import numpy as np
from sklearn.neural_network import MLPClassifier

In [37]:
def CulturalAlgorithm(X, y, population_size, belief_space_size, num_generations, mutation_probability, selection_pressure):
    """
    X: input features
    y: target variable
    population_size: size of the population
    belief_space_size: size of the belief space
    num_generations: number of generations
    mutation_probability: probability of mutation
    selection_pressure: selection pressure parameter
    """
    
    # Define the neural network model
    model = MLPClassifier(hidden_layer_sizes=(6, 6), activation='relu', solver='adam', max_iter=30)
    
    # Define the population and belief space
    population = [model.coefs_, model.intercepts_]
    belief_space = [population] * belief_space_size
    
    # Perform the optimization
    for generation in range(num_generations):
        
        # Generate new individuals through mutation and crossover
        new_population = []
        for individual in population:
            new_individual = []
            for param in individual:
                if np.random.rand() < mutation_probability:
                    mutation = np.random.randn(*param.shape)
                    new_param = param + mutation
                else:
                    new_param = param
                new_individual.append(new_param)
            new_population.append(new_individual)
        
        # Evaluate the fitness of the new population
        fitness = []
        for individual in new_population:
            model.coefs_, model.intercepts_ = individual
            y_pred = model.predict(X)
            fitness.append((individual, np.sum(y_pred == y)))
        
        # Sort the population by fitness
        fitness_sorted = sorted(fitness, key=lambda x: x[1], reverse=True)
        
        # Update the belief space with the best individuals
        best_individuals = [x[0] for x in fitness_sorted[:selection_pressure]]
        belief_space.append(best_individuals)
        belief_space = belief_space[-belief_space_size:]
        
        # Select the next generation from the belief space
        population = []
        for i in range(population_size):
            belief = np.random.choice(belief_space)
            individual = [np.copy(param) for param in np.random.choice(belief)]
            population.append(individual)
    
    # Return the best individual
    return fitness_sorted[0][0]

In [38]:
def cultural_algorithm(X_train, y_train, X_test, y_test):
    """
    X_train: training input features
    y_train: training target variable
    X_test: test input features
    y_test: test target variable
    """
    # Define the hyperparameters
    population_size = 50
    belief_space_size = 10
    num_generations = 100
    mutation_probability = 0.1
    selection_pressure = 10
    
    # Run the CulturalAlgorithm
    best_individual = CulturalAlgorithm(X_train, y_train, population_size, belief_space_size, num_generations, mutation_probability, selection_pressure)
    
    # Define the neural network model with the best parameters
    model = MLPClassifier(hidden_layer_sizes=(6, 6), activation='relu', solver='adam', max_iter=30)
    model.coefs_, model.intercepts_ = best_individual
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    accuracy = np.sum(y_pred == y_test) / len(y_test)
    
    return accuracy

In [40]:
acc = cultural_algorithm(X_train,y_train,X_test,y_test)
print(acc)

0.91
